In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import chromedriver_binary
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re

url = "https://spi21.mapyourshow.com/8_0/floorplan/"

In [2]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import chromedriver_binary
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

url = 'https://spi21.mapyourshow.com/8_0/explore/exhibitor-alphalist.cfm#/'
options = webdriver.ChromeOptions()
options.add_argument('headless')  
driver = webdriver.Chrome()
driver.get(url)

time.sleep(3)

#need to scroll down
driver.execute_script("window.scrollTo(20, 40);")

time.sleep(5)

#click see all results
idName = "exhibitor-results"
# xPath = "//section[@id = '%s']/div/span[1]"%(idName)
xPath = "//*[@id = '%s']/div[1]/span[1]/a/span"%(idName)
#wait until there is any element to hover
wait = WebDriverWait(driver, 20)
auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, xPath)))
element = driver.find_element_by_xpath(xPath)
element.click()

time.sleep(5)

xPath = '//a'
elements = driver.find_elements_by_xpath(xPath)
#why page content is not the same after selenium?
hrefList = [i.get_attribute("href") for i in elements]

nameList = [i.get_attribute("innerText") for i in elements]


#result
resultList = []
for no, href in enumerate(hrefList):
    if re.search("exhid=\d+$", str(href)):
        resultList.append([nameList[no], href])
print(len(resultList))

driver.close()

472


In [8]:
finalHrefList = set([i[1] for i in resultList])

filename = 'C:/Users/JiunShyanGoh/Documents/Upwork/DataScraping/Data/HrefList.csv'

with open(filename, 'w') as filehandle:
    for href in finalHrefList:
        filehandle.write('%s\n' %href)

In [11]:
filename = 'C:/Users/JiunShyanGoh/Documents/Upwork/DataScraping/Data/HrefList.csv'

a = []
with open(filename, 'r') as filehandle:
    for line in filehandle:
        a.append(re.sub('\n','',line))

In [13]:
url = 'https://spi21.mapyourshow.com/8_0/exhibitor/exhibitor-details.cfm?exhid=8799734'
options = webdriver.ChromeOptions()
options.add_argument('headless')  
driver = webdriver.Chrome(options = options)
driver.get(url)

In [14]:
xPath = "//div[@class = 'wrapper-centered']"
elements = driver.find_elements_by_xpath(xPath)

In [16]:
[i.text for i in elements]

['Contact Us\nCONTACT US\nAddress: 610 A East Battlefield Road\nAddress 2: PMB 247\nCity: Springfield\nState: MO\nCountry: United States of America\nZip: 65807\nPhone: 5415161100\nWebsite: www.cascadeecominerals.com',
 'Booths\nFirst Floor Exhibit Hall — 2171',
 'About Cascade Eco Minerals\nCascade Eco Minerals utilizes state-of-the-art technology, innovative processes, and proprietary offerings to provide a comprehensive domestic recycling solution for the Solar Industry. With over 80 years of experience in glass processing and a mission to grow the circular economy, you’ll find no greater partner in solar glass recycling, than CEM.',
 'Notes\nCreate or Log in to My Show Planner to add notes.']

In [29]:
url = 'https://spi21.mapyourshow.com/8_0/exhibitor/exhibitor-details.cfm?exhid=7088'
# url = 'https://spi21.mapyourshow.com/8_0/exhibitor/exhibitor-details.cfm?exhid=8799734'
options = webdriver.ChromeOptions()
options.add_argument('headless')  
driver = webdriver.Chrome(options = options)
driver.get(url)

#need to scroll down
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

[i.text for i in driver.find_elements_by_xpath("//h2/a")]

['Manufacturer - PV Cell', 'Manufacturer - PV Module']

In [28]:
''.join([i.text for i in driver.find_elements_by_xpath("//h2/a")])

''

In [ ]:
#number of seconds to sleep
timeSleep = 3

#get usernamelist and name list 
options = webdriver.ChromeOptions()
options.add_argument('headless')  
driver = webdriver.Chrome()
driver.get(url)

#wait for canvas of floor to show up
canvasClassName = "ol-unselectable"
xPath = "//canvas[@class = '%s']"%canvasClassName
#wait until there is any element to hover
wait = WebDriverWait(driver, 20)
auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, xPath)))

# clickableElement
#with position (not working yet)
leftPosition = "139px"
topPosition = "179px"
styleValue = "position: absolute; left: %s; top: %s"%(leftPosition, topPosition)
className = "ol-overlay-container ol-selectable"
xPath = "//div[@class = '%s']"%className +  "[@style = '%s"%styleValue + \
"; display: none;']"

#this work
className = 'mys-floorplan-canvas'
xPath = "//div[@class = '%s']"%className
clickableElement = driver.find_element_by_xpath(xPath)
action = ActionChains(driver)
action.move_to_element(clickableElement).click().perform()


#company name
resultDict = {}

className = 'flex  items-center  mb3  mb0-m  flex-wrap'
xPath = "//div[@class = '%s']"%className
auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, xPath)))
element = driver.find_element_by_xpath(xPath)
#alternative: companyName = element.text
companyName = element.get_attribute("innerText")
resultDict['Name'] = companyName

#address, city, state, country, zip, phone, website
elements = driver.find_elements_by_xpath("//div[@class = 'wrapper-centered']")
textList = [element.text for element in elements]
#fields in the first element of textList
fieldList = ['Address', 'City', 'State', 'Country', 'Zip', 'Phone', 'Website']
for field in fieldList:
    resultDict[field] = re.search("%s:(.*)\n*"%field, textList[0]).group(1).strip()
        
#booth
resultDict['Booth'] = textList[1].split('\n')[1]

#about
resultDict['About'] = textList[2].split('\n')[1]

#product categories
resultDict['Product Categories'] = textList[3].split('\n')[1:]

In [ ]:
companyName

In [ ]:
#number of seconds to sleep
timeSleep = 3

#get usernamelist and name list 
options = webdriver.ChromeOptions()
options.add_argument('headless')  
driver = webdriver.Chrome()
driver.get(url)

#wait for canvas of floor to show up
canvasClassName = "ol-overlaycontainer-stopevent"
xPath = "//div[@class = '%s']"%canvasClassName
#wait until there is any element to hover
wait = WebDriverWait(driver, 20)
auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, xPath)))

leftPosition = "547px"
topPosition = "377px"
styleValue = "position: absolute; left: %s; top: %s;"%(leftPosition, topPosition)
className = "ol-overlay-container ol-selectable"
xPath = "//div[@class = '%s']"%className +  "[@style = '%s']"%styleValue 
auto_element = wait.until(EC.visibility_of_element_located((By.XPATH, xPath)))
clickableElement = driver.find_element_by_xpath(xPath)
action = ActionChains(driver)
action.move_to_element(clickableElement).click().perform()

In [ ]:
xPath